In [12]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
import nltk
import pprint as pp
import db_scripts
import pprint

In [13]:
def load_stop_words(stop_word_file):
    stop_words = []
    fo = open(stop_word_file)
    for line in fo:
        if line.strip()[0:1] != "#":
            for word in line.split():  # in case more than one per line
                stop_words.append(word)
    fo.close()
    return stop_words

In [39]:
def load_keywords_from_bucket(keyword_file):
    word_map = dict()
    file = open(keyword_file)
    for line in file:
        key = line.split(':')
        #print(key)
        x, y = Keywords(key[1]).fetch()
        if key[0] in word_map:
            val = word_map[key[0]]
            val.append(y)
            word_map[key[0]] = val
        else:
            val = list()
            val.append(y)
            word_map[key[0]] = val
    pprint.pprint(word_map)
    file.close()
load_keywords_from_bucket("keywords.txt")

{'Data Science': [['data',
                   'mine',
                   'machin',
                   'learn',
                   'data',
                   'analyt',
                   'mine',
                   'analyt',
                   'python',
                   'tableau',
                   'sa',
                   'visual',
                   'regress',
                   'classif',
                   'cluster',
                   'supervis',
                   'unsupervis',
                   'artifici',
                   'intellig']],
 'Hardware': [['archotectur',
               'micro',
               'design',
               'asic',
               'parallel',
               'microprocessor',
               'microcontrol',
               'assembl']],
 'Networking': [['network',
                 'lan',
                 'switch',
                 'protocol',
                 'internet',
                 'wireless',
                 'tcp',
                 'commun',
        

In [15]:
def remove_puncts(text):
    tokenizer = RegexpTokenizer(r'\w+')
    words_no_punct = tokenizer.tokenize(text)
    data = ' '.join(words_no_punct)
    return data

In [16]:
def is_number(s):
    try:
        float(s) if '.' in s else int(s)
        return True
    except ValueError:
        return False

In [17]:
def generate_candidate_keywords(data, stop_words):
    words = [w for w in word_tokenize(
        data) if w.lower() not in stop_words and not is_number(w)]
    return words

In [18]:
def stem_lemmatize(keywords):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    tokenizer = RegexpTokenizer(r'\w+')
#     table.column_headers = ["Actual", "Lemmatize", "Stem"]
    x = []
    y = []
    for word in keywords:
        x.append(stemmer.stem(word))
        y.append(lemmatizer.lemmatize(word))
    return x

In [19]:
alda = "Introduction to the problems and techniques for automated discovery of knowledge in databases. Topics include representation, evaluation, and formalization of knowledge for discovery; classification, prediction, clustering, and association methods.Selected applications in commerce, security, and bioinformatics. Students cannot get credit for both CSC 422 and CSC 522."

In [20]:
class Keywords:
    def __init__(self, text):
        self.text = text
        self.stop_words = load_stop_words("FoxStopList.txt")

    def fetch(self):
        candidate = remove_puncts(self.text)
        keywords = generate_candidate_keywords(candidate, self.stop_words)
        # table = stem_lemmatize(keywords)
        st = stem_lemmatize(keywords)
        return keywords, st

In [21]:
l, p = Keywords(alda).fetch()
print(p)
print(l)

['autom', 'discoveri', 'knowledg', 'databas', 'represent', 'evalu', 'formal', 'knowledg', 'discoveri', 'classif', 'predict', 'cluster', 'associ', 'method', 'applic', 'commerc', 'secur', 'bioinformat']
['automated', 'discovery', 'knowledge', 'databases', 'representation', 'evaluation', 'formalization', 'knowledge', 'discovery', 'classification', 'prediction', 'clustering', 'association', 'methods', 'applications', 'commerce', 'security', 'bioinformatics']


In [22]:
set(p)

{'applic',
 'associ',
 'autom',
 'bioinformat',
 'classif',
 'cluster',
 'commerc',
 'databas',
 'discoveri',
 'evalu',
 'formal',
 'knowledg',
 'method',
 'predict',
 'represent',
 'secur'}

In [23]:
p = set(p)

In [24]:
from collections import defaultdict
d = defaultdict()
for i in p:
    if i in d:
        d[i].append('CSC522')
    else:
        d[i] = ['CSC522']

In [25]:
d

defaultdict(None,
            {'applic': ['CSC522'],
             'associ': ['CSC522'],
             'autom': ['CSC522'],
             'bioinformat': ['CSC522'],
             'classif': ['CSC522'],
             'cluster': ['CSC522'],
             'commerc': ['CSC522'],
             'databas': ['CSC522'],
             'discoveri': ['CSC522'],
             'evalu': ['CSC522'],
             'formal': ['CSC522'],
             'knowledg': ['CSC522'],
             'method': ['CSC522'],
             'predict': ['CSC522'],
             'represent': ['CSC522'],
             'secur': ['CSC522']})

In [26]:
import pandas as pd
df = pd.DataFrame().from_dict(d)

In [27]:
df

,applic,associ,autom,bioinformat,classif,cluster,commerc,databas,discoveri,evalu,formal,knowledg,method,predict,represent,secur
0,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522


In [28]:
d['predict'].append('CSC501')
df = pd.DataFrame().from_dict(d,orient='index').transpose()
df

,evalu,classif,associ,commerc,autom,formal,applic,bioinformat,cluster,databas,discoveri,method,secur,represent,knowledg,predict
0,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,CSC501


In [29]:
import db_scripts
all_courses = db_scripts.db_fetch_all("wolfpal", "courses")

Authentication Successful.


In [30]:
from collections import defaultdict
d = defaultdict()

In [31]:
for course in all_courses:
    l, p = Keywords(course['desc']).fetch()
    # print(course['desc'])
    p = set(p)
    for i in p:
        if i in d:
            d[i].append(course['branch']+course['number'])
        else:
            d[i] = [(course['branch']+course['number'])]

In [32]:
d

defaultdict(None,
            {'AI': ['CSC520'],
             'analysi': ['CSC510'],
             'artifici': ['CSC520'],
             'associ': ['CSC520'],
             'avl': ['CSC541'],
             'bit': ['CSC541'],
             'bloom': ['CSC541'],
             'chang': ['CSC510'],
             'characterist': ['CSC541'],
             'code': ['CSC510', 'CSC541'],
             'collis': ['CSC541'],
             'complex': ['CSC541'],
             'configur': ['CSC510'],
             'control': ['CSC510'],
             'coordin': ['CSC501'],
             'cost': ['CSC510'],
             'cycl': ['CSC510'],
             'data': ['CSC541'],
             'databas': ['CSC520'],
             'deadlock': ['CSC501'],
             'design': ['CSC501', 'CSC510', 'CSC541'],
             'develop': ['CSC510', 'CSC541'],
             'disjoint': ['CSC541'],
             'distribut': ['CSC501'],
             'dynam': ['CSC541'],
             'effect': ['CSC541'],
             'effici': ['CSC54

In [33]:
import pandas as pd
df = pd.DataFrame().from_dict(d,orient='index').transpose()

In [34]:
df

,system,element,design,distribut,manag,deadlock,relat,schedul,memori,issu,...,program,prove,techniqu,represent,artifici,databas,intellig,natur,knowledg,space
0,CSC501,CSC501,CSC501,CSC501,CSC501,CSC501,CSC501,CSC501,CSC501,CSC501,...,CSC520,CSC520,CSC520,CSC520,CSC520,CSC520,CSC520,CSC520,CSC520,CSC520
1,CSC520,CSC520,CSC510,None,CSC510,None,CSC541,CSC510,None,None,...,None,None,None,None,None,None,None,None,None,None
2,None,None,CSC541,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [35]:
usersays = "I like AI and systems"
p,l = Keywords(usersays).fetch()
print(l)

['AI', 'system']


In [36]:
for i in l:
    if i in d:
        print("Yeh course le: ",d[i])

Yeh course le:  ['CSC520']
Yeh course le:  ['CSC501', 'CSC520']


In [42]:
s = set()
s.add(1)
s.add(2)
s.add(3)

In [43]:
l = list(s)

In [ ]:
l